# Getting started with `pyrenew`

This notebook illustrates two features of `pyrenew`: (a) the set of included `RandomProcess`es, and (b) model composition.

## Hospitalizations model

`pyrenew` has four main components:

- Utilitiy and math functions
- The `processes` sub-module
- The `observations` sub-module
- The `models` sub-module

Both `processes` and `observations` contain classes that inherit from the meta class `RandomProcess`. The classes under `models` inherit from the meta class `Model`.

```{mermaid}
flowchart TB

    subgraph randprocmod["Processes module"]
        direction TB
        simprw["SimpleRandomWalkProcess"]
        rtrw["RtRandomWalkProcess"]        
    end

    subgraph obsmod["Observations module"]
        direction TB
        pois["PoissonObservation"]
        hosp_obs["HospitalizationsObservation"]
    end

    rp(("RandomProcess")) --> |Inherited by| simprw
    rp -->|Inherited by| rtrw
    rp -->|Inherited by| pois
    rp -->|Inherited by| hosp_obs


    model(("Model")) -->|Inherited by| basic["BasicRenewalModel"]

    simprw -->|Composes| rtrw
    rtrw -->|Composes| basic
    basic -->|Inherited by| hosp["HospitalizationsModel"]


    pois -->|Composes| hosp
    hosp_obs -->|Composes| hosp

    %% Metaclasses
    classDef Metaclass color:black,fill:white
    class rp,model Metaclass

    %% Random process
    classDef Randproc fill:purple,color:white
    class rtrw,simprw Randproc

    %% Models
    classDef Models fill:teal,color:white
    class basic,hosp Models
```